In [ ]:
from tkinter import *
import datetime
number = 0

import cv2

import tensorflow as tf

import numpy as np
import time
import pandas as pd
from time import sleep

import urllib.request
import requests
import threading
import json

import random


READ_API_KEY='5ZU70EK3OFZMAOGS'
CHANNEL_ID= '1751724'



from time import sleep
import RPi.GPIO as GPIO

DIR = 20       # Direction GPIO Pin
STEP = 21      # Step GPIO Pin
CW = 1         # Clockwise Rotation
CCW = 0        # Counterclockwise Rotation
SPR = 100       # Steps per Revolution (360 / 1.8)

GPIO.setmode(GPIO.BCM)
GPIO.setup(DIR, GPIO.OUT)
GPIO.setup(STEP, GPIO.OUT)
GPIO.output(DIR, CW)
from adafruit_servokit import ServoKit
kit = ServoKit(channels=16)

MODE = (14, 15, 18) # Microstep Resolution GPIO Pins
GPIO.setup(MODE, GPIO.OUT)
RESOLUTION = {'Full': (0, 0, 0),
              'Half': (1, 0, 0),
              '1/4': (0, 1, 0),
              '1/8': (1, 1, 0),
              '1/16': (0, 0, 1),
              '1/32': (1, 0, 1)}

GPIO.output(MODE, RESOLUTION['Full'])

step_count = SPR * 10
delay = 0.001 #.005 / 32

    

modelname="cottonvgg16.h5"

model=tf.keras.models.load_model(modelname)
cap=cv2.VideoCapture(0)


window = Tk()
window.title("Programme")
window.geometry('900x700')


# set window size

#set window color
window['background']='#305349'


window.resizable(False, False)


label = Label(window, text=number)
label.place(x=600,y=500)

def up():
    for x in range(step_count):
        GPIO.output(STEP, GPIO.HIGH)
        sleep(delay)
        GPIO.output(STEP, GPIO.LOW)
        sleep(delay)
    sleep(1)
    GPIO.output(DIR, CCW)
    


def clicked1():
    global number
    
    number = "Move UP" 
    label.config(text=number,font=('Helvetica bold', 26))
    t3= threading.Thread(target=up, args=())
    t3.start()
    


    
    

button1 = Button(window, text="UP", command=clicked1,height= 5, width=10)
button1.place(x=600, y=50)


def clicked5():
    global number
    number = "Auto" 
    label.config(text=number,font=('Helvetica bold', 26))

    

button5 = Button(window, text="Auto", command=clicked1,height= 5, width=10)
button5.place(x=600, y=200)

def left():
    kit.servo[0].angle = 180
    time.sleep(1)
    kit.servo[0].angle = 89
    

def clicked2():
    global number
    number ="Move Left"
    label.config(text=number,font=('Helvetica bold', 26))
    t4= threading.Thread(target=left, args=())
    t4.start()
    

    

button2 = Button(window, text="LEFT", command=clicked2,height= 5, width=10)
button2.place(x=400, y=200)

def down():
    for x in range(step_count):
        GPIO.output(STEP, GPIO.HIGH)
        sleep(delay)
        GPIO.output(STEP, GPIO.LOW)
        sleep(delay)


def clicked3():
    global number
    number =  "Move Down"
    label.config(text=number,font=('Helvetica bold', 26))
    t5=threading.Thread(target=right, args=())
    t5.start()
    

import threading
    

button3 = Button(window, text="DOWN", command=clicked3,height= 5, width=10)
button3.place(x=600, y=350)

def right():
    kit.servo[0].angle = 0
    time.sleep(1)
    kit.servo[0].angle = 89
    
    
def clicked4():
    global number
    number = "Move Right" 
    label.config(text=number,font=('Helvetica bold', 26))
    t6= threading.Thread(target=right, args=())
    t6.start()


button4 = Button(window, text="RIGHT", command=clicked4,height= 5, width=10)
button4.place(x=800, y=200)


def update_clock():
    # get current time as text
    current_time = datetime.datetime.now().strftime("Time: %H:%M:%S")
    t2= threading.Thread(target=rece, args=())
    t2.start()
    
    # udpate text in Label
    lab.config(text=current_time)
    #lab['text'] = current_time
    
    # run itself again after 1000 ms
    window.after(1000, update_clock) 
    
def output():
    try:
        TS = urllib.request.urlopen("http://api.thingspeak.com/channels/%s/feeds/last.json?api_key=%s" \
                       % (CHANNEL_ID,READ_API_KEY))
        response = TS.read()
        data=json.loads(response)
        data1 = data['field4']
        if data1==1:
            for x in range(step_count):
                GPIO.output(STEP, GPIO.HIGH)
                sleep(delay)
                GPIO.output(STEP, GPIO.LOW)
                sleep(delay)
    except:
        
        pass
    
    TS.close()
    lab3.config(text="iot"+str(data1),font=('Helvetica bold', 26))
    window.after(2000, output) 

lab3 = Label(window)
lab3.place(x=400, y=300)

def videocap():
    # get current time as text
    b,imgoriginal=cap.read()
    img1=np.asarray(imgoriginal)
    img1=np.resize(img1,(224,224,3))
    img=img1.reshape(1,224,224,3)
    
    predictions=model.predict(img)
    predict_x=model.predict(img)
    classindex=np.argmax(predict_x,axis=1)
    probabilityvalue=np.amax(predictions)
    if probabilityvalue>0.1:
        try:
            if classindex==0:
                baseURL = 'https://api.thingspeak.com/update?api_key=3VTBXO0Q99V2SEGH&field1=1'
                urllib.request.urlopen(baseURL)
                data="fresh"
            if classindex==1:
                baseURL = 'https://api.thingspeak.com/update?api_key=3VTBXO0Q99V2SEGH&field2=1'
                urllib.request.urlopen(baseURL)
                data="becterial"
            if classindex==2:
                baseURL = 'https://api.thingspeak.com/update?api_key=3VTBXO0Q99V2SEGH&field3=1'
                urllib.request.urlopen(baseURL)
                data="curl"
        except:
            print("internet issue")
    lab1.config(text=data,font=('Helvetica bold', 26))

def rece():
    
    while True:
        try:
            TS = urllib.request.urlopen("http://api.thingspeak.com/channels/%s/feeds/last.json?api_key=%s" \
                       % (CHANNEL_ID,READ_API_KEY))
            response = TS.read()
            data=json.loads(response)
            data1 = data['field4']
            print(data1)
            if data1=="1":
                clicked1()
            elif data=="2":
                clicked3()
                
            elif data=="3":
                 clicked2()
                
            elif data=="4":
                clicked4()
            
        except:
            pass
    
    
def detect():
    t1= threading.Thread(target=videocap, args=())
    t1.start()
    
    # udpate text in Label
    #lab['text'] = current_time
    
    # run itself again after 1000 ms
    window.after(10000, detect) 


lab = Label(window)
lab.pack()

lab1 = Label(window)
lab1.place(x=200, y=300)

update_clock()
detect()
output()

window.mainloop()